# Example: Vis-pump vis-probe on Corroles, analyzed with skultrafast.dataset

skultrafast is a package mainly for working with time-resolved spectra. The notebook shows how to
use the new dataset-class to work with such a spectra and to fit it with a exponetial model. First we import
the necessary modules and configure matplotlib to show inline graphics.

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('../..')

In [2]:
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['figure.dpi'] = 150
from skultrafast.dataset import DataSet
import skultrafast
from skultrafast import data_io
print(skultrafast.__file__)
skultrafast.__version__

SyntaxError: invalid syntax (plot_helpers.py, line 210)

In [ ]:
askultrafast.__path__

In [ ]:
a = np.load(skultrafast.__path__[0] + '\\examples\\test.npz')
wl, data, t = a['wl'], a['data'], a['t']

## Loading data
Next, we will load a dataset in textform using `numpy`. The file we load is a table where each column (expect the first which contains the delay times) is the absorbance change of one channel. The first row contains the frequency in wavelengths. We extract seperate arrays for the data, the wavelengths and the delaytimes.

In [ ]:
a = np.load(skultrafast.__path__[0] + '\\examples\\test.npz')
wl, data, t = a['wl'], a['data']/3., a['t']*1000-2
data_io.save_txt('corrole_ex', wl, t, data)
#t = d[1:, 0]
#wl = d[0, 1:]
#data = d[1:, 1:]

Lets look at the constructor of the `DataSet` class:

In [ ]:
help(DataSet.__init__)

As we see, we have all the necessay variables, additionlly we can name the dataset. Since the `freq_unit` defaults to 'nm' we don't need to supply this argument.

In [ ]:
ds = DataSet(wl, t, data, name="Al(tpfc)(py)_2")

## Overview

To get an general idea of the transient spectra, it is best to plot some data.  All plotting functions are in the `DataSet.plot` object, which is an instance of `DataSetPlotter`. The plotting functions are using the wavelength scale by default. This can be changed by changing the `freq_unit` of the `DataSetPlotter` object. 

In [ ]:
ds.plot.freq_unit = 'nm' # does nothing, since 'nm' is the default
# ds.plot.freq_unit = 'cm' would use wavenumbers

First, we want to check if the dataset is corrected for dispersion. For that we plot a colormap around the time-zero. 

In [ ]:
ds.plot.map(symlog=0, con_step=10., con_filter=(3, 10))
plt.ylim(-2, 2)

Evidently, the dataset is not corrected for dispersion. Since it is easier to work with a dispersion corrected dataset, we try to get an estimate of the dispersion using the data directly. 

*skultrafast* does this by first using a simple heuristic for determining the time-zero for each transient. The resulting dispersion curve is then fitted with a polynomial, using a robust fitting method. More details are given in the documentation.

In [ ]:
# First calculate and plot the estimate.
res = ds.estimate_dispersion(heuristic_args=(2,), deg=3)
ds.plot.map(symlog=0, con_step=10., con_filter=(3, 10))
plt.ylim(-2, 2)
plt.plot(ds.wl, res.polynomial(ds.wl))
plt.plot(ds.wl, res.tn)

By default, *skultrafast* uses a very simple heuristic to find the time-zero. It looks for the earliest value above a given limit in each transient, and therefore underestimates the time-zero systematically. Therefore we slightly shift the time-zero. 

This generallay works surprinsingly well. But if the exact time-zero is necessary, I recommend to try other methods or measure the dispersion directly.

**WARNING** The cell below changes the dataset inplace. Therefore repeated calls to the cell will shift the time-zero even more.

In [ ]:
new_ds = res.correct_ds #warning, this is no copy
new_ds.t -= 0.1 

In [ ]:
new_ds.plot.map(con_step=10., con_filter=(3, 5))

In [ ]:
res.correct_ds.plot.spec([-.1, 0.05, 0.3, 1, 2, 150])

In [ ]:
res.correct_ds.plot.trans([500, 620, 680], symlog=1)
